# Segmenting and Clustering Neighborhoods in Toronto

Week 3 Coursera Applied Data Science Capstone


##  Question 1: Transform the data in the table on the Wikipedia page into a pandas dataframe.

In [84]:
import urllib.request
import pandas as pd
import geocoder
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

In [15]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#url = "https://en.wikipedia.org/wiki/1999%E2%80%932000_FA_Premier_League"

In [16]:
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [17]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [18]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")
all_tables

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B


In [19]:
right_table=soup.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>

In [53]:
A=[]
B=[]
C=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
       

In [54]:
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
df

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
...,...,...,...
175,M5Z\n,Not assigned\n,Not assigned\n
176,M6Z\n,Not assigned\n,Not assigned\n
177,M7Z\n,Not assigned\n,Not assigned\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


### The elemnte in the table have \n in the end, we don't want it, so we eliminate de \n of every elemnt .
 

##### PostalCode

In [55]:
print(A)
D=[]
for i in A: 
    D.append(i[:-1])
print(D)



['M1A\n', 'M2A\n', 'M3A\n', 'M4A\n', 'M5A\n', 'M6A\n', 'M7A\n', 'M8A\n', 'M9A\n', 'M1B\n', 'M2B\n', 'M3B\n', 'M4B\n', 'M5B\n', 'M6B\n', 'M7B\n', 'M8B\n', 'M9B\n', 'M1C\n', 'M2C\n', 'M3C\n', 'M4C\n', 'M5C\n', 'M6C\n', 'M7C\n', 'M8C\n', 'M9C\n', 'M1E\n', 'M2E\n', 'M3E\n', 'M4E\n', 'M5E\n', 'M6E\n', 'M7E\n', 'M8E\n', 'M9E\n', 'M1G\n', 'M2G\n', 'M3G\n', 'M4G\n', 'M5G\n', 'M6G\n', 'M7G\n', 'M8G\n', 'M9G\n', 'M1H\n', 'M2H\n', 'M3H\n', 'M4H\n', 'M5H\n', 'M6H\n', 'M7H\n', 'M8H\n', 'M9H\n', 'M1J\n', 'M2J\n', 'M3J\n', 'M4J\n', 'M5J\n', 'M6J\n', 'M7J\n', 'M8J\n', 'M9J\n', 'M1K\n', 'M2K\n', 'M3K\n', 'M4K\n', 'M5K\n', 'M6K\n', 'M7K\n', 'M8K\n', 'M9K\n', 'M1L\n', 'M2L\n', 'M3L\n', 'M4L\n', 'M5L\n', 'M6L\n', 'M7L\n', 'M8L\n', 'M9L\n', 'M1M\n', 'M2M\n', 'M3M\n', 'M4M\n', 'M5M\n', 'M6M\n', 'M7M\n', 'M8M\n', 'M9M\n', 'M1N\n', 'M2N\n', 'M3N\n', 'M4N\n', 'M5N\n', 'M6N\n', 'M7N\n', 'M8N\n', 'M9N\n', 'M1P\n', 'M2P\n', 'M3P\n', 'M4P\n', 'M5P\n', 'M6P\n', 'M7P\n', 'M8P\n', 'M9P\n', 'M1R\n', 'M2R\n', 'M3R\n', 

##### Borough

In [71]:
E=[]
for i in B: 
    E.append(i[:-1])
print(E)

['Not assigned', 'Not assigned', 'North York', 'North York', 'Downtown Toronto', 'North York', 'Downtown Toronto', 'Not assigned', 'Etobicoke', 'Scarborough', 'Not assigned', 'North York', 'East York', 'Downtown Toronto', 'North York', 'Not assigned', 'Not assigned', 'Etobicoke', 'Scarborough', 'Not assigned', 'North York', 'East York', 'Downtown Toronto', 'York', 'Not assigned', 'Not assigned', 'Etobicoke', 'Scarborough', 'Not assigned', 'Not assigned', 'East Toronto', 'Downtown Toronto', 'York', 'Not assigned', 'Not assigned', 'Not assigned', 'Scarborough', 'Not assigned', 'Not assigned', 'East York', 'Downtown Toronto', 'Downtown Toronto', 'Not assigned', 'Not assigned', 'Not assigned', 'Scarborough', 'North York', 'North York', 'East York', 'Downtown Toronto', 'West Toronto', 'Not assigned', 'Not assigned', 'Not assigned', 'Scarborough', 'North York', 'North York', 'East York', 'Downtown Toronto', 'West Toronto', 'Not assigned', 'Not assigned', 'Not assigned', 'Scarborough', 'North

##### Neighborhood

In [57]:
F=[]
for i in C: 
    F.append(i[:-1])

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [68]:
for i in range(len(F)):

    if F[i]== 'Not assigned':
        F[i]= E[i]
 


### Make the dataframe

In [90]:

df=pd.DataFrame(D,columns=['Postal Code'])
df['Borough']=E
df['Neighborhood']=F

In [91]:
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### Ignore cells with a borough that is Not assigned.

In [72]:
df = df.set_index("Borough")
df = df.drop('Not assigned', axis=0) # Delete all rows with label "Ireland"


### The dataframe

In [73]:
df


,PostalCode,Neighborhood
Borough,,
North York,M3A,Parkwoods
North York,M4A,Victoria Village
Downtown Toronto,M5A,"Regent Park, Harbourfront"
North York,M6A,"Lawrence Manor, Lawrence Heights"
Downtown Toronto,M7A,"Queen's Park, Ontario Provincial Government"
...,...,...
Etobicoke,M8X,"The Kingsway, Montgomery Road, Old Mill North"
Downtown Toronto,M4Y,Church and Wellesley
East Toronto,M7Y,"Business reply mail Processing Centre, South C..."


##### Shape

In [74]:
df.shape

(103, 2)

## Question 2: Get the latitude and the longitude coordinates of each neighborhood.

In [88]:
Latitudes=pd.read_csv('Geospatial_Coordinates.csv')

In [89]:
Latitudes

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [97]:
Coor = pd.merge(df, Latitudes, on='Postal Code')
Coor

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Question 3: Explore and cluster the neighborhoods in Toronto.

In [98]:
address = 'Toronto, TOR'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7627912, -79.4064452.


In [96]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

TypeError: string indices must be integers